# TITLE

We downloaded the files from: <br>
accidentalidad.csv: http://medata.gov.co/dataset/accidentalidad 
<br>
AccidentalidadgeoreferenciadaYYYY:
<br>
http://medata.gov.co/dataset/accidentalidad-geo-referenciada-2019
http://medata.gov.co/dataset/accidentalidad-geo-referenciada-2018
http://medata.gov.co/dataset/accidentalidad-geo-referenciada-2017
http://medata.gov.co/dataset/accidentalidad-geo-referenciada-2016
http://medata.gov.co/dataset/accidentalidad-geo-referenciada-2015
http://medata.gov.co/dataset/accidentalidad-geo-referenciada-2014
<br>

To ingest about dataset we decided to use two different dataframes, accidentalidad dataframe ```a_df``` to aggregate the differents files by year and include the incident time wich is not included in the accidentalidad_georrererenciada dataframe ```ag_df```, so we use these two dataframes to verify the integrity of the datasets and build one dataset with all the information we need.

In the following code we clean and merge the datasets:

# Download all the data


In [2]:
import requests
import pandas as pd
from ast import literal_eval
from unidecode import unidecode
pd.options.display.max_columns = None





In [ ]:

urls = {'url_accidentalidad' : 'http://medata.gov.co/sites/default/files/medata_harvest_files/accidentalidad.csv',
'url_accidentalidad_georreferenciada_2019' : 'https://opendata.arcgis.com/datasets/e132c88e875b429db4c1a3c80b7977da_20.csv',
'url_accidentalidad_georreferenciada_2018' : 'https://opendata.arcgis.com/datasets/e4b4645cc47b462495305db0574982b5_16.csv',
'url_accidentalidad_georreferenciada_2017' : 'https://opendata.arcgis.com/datasets/b7f7158734f64e7f849df6ac4a6b10f1_12.csv',
'url_accidentalidad_georreferenciada_2016' : 'https://opendata.arcgis.com/datasets/a65c3aff0ef34973a2441b6cd0fbc24a_8.csv',
'url_accidentalidad_georreferenciada_2015' : 'https://opendata.arcgis.com/datasets/025f3be66fbd48b888b779cf00928ae8_4.csv',
'url_accidentalidad_georreferenciada_2014' : 'https://opendata.arcgis.com/datasets/505e89d2ade143a684d51b60236ba285_0.csv',
'url_accidentalidad_motos':'https://www.datos.gov.co/api/views/b7ik-2upt/rows.csv?accessType=DOWNLOAD',
'url_camaras_cctv' : 'https://opendata.arcgis.com/datasets/9721295ac3644bfbb076795b36912037_9.csv',
'url_camaras_fotodeteccion' : 'https://opendata.arcgis.com/datasets/bfcd5e4e3e364957b1aeeff0ee022815_7.csv',
'url_cruces_semafóricos' : 'https://opendata.arcgis.com/datasets/072ed4e620b849d6bdbb058c3cb77922_0.csv',
'url_limite_comunas' : 'https://opendata.arcgis.com/datasets/283d1d14584641c9971edbd2f695e502_6.csv',
'url_limite_barrios' : 'https://opendata.arcgis.com/datasets/1a6dbf15865b4357aa559699ea90c5b9_7.csv'}

for key in urls:
    r =requests.get(urls[key], allow_redirects=True)
    open(key.replace('url_','')+'.csv', 'wb').write(r.content)

### Concatenate all the accidentalidad_georreferenciada files

In [149]:
# ag_df: yearly file
ag_df = pd.DataFrame()
path = "./"
for  file in os.listdir(path):
    if 'accidentalidad_georreferenciada_' in file:
        df_tmp = pd.read_csv(os.path.join(path, file))
        df_tmp.columns = [c.lower() for c in df_tmp.columns]
        df_tmp['file'] = file
        ag_df = pd.concat([ag_df, df_tmp])

ag_df.to_csv("../raw/accidentalidad_georreferenciada.csv",index=False)

### Check the columns

# Delete 'radicados' duplicates from three sources:
- accidentalidad.csv
- accidentalidad_georreferenciada.csv
- accidentalidad_motos.csv

We found some duplicates values in "radicado" field so we procedure to drop them.


In [58]:
ag_df = pd.read_csv("../raw/accidentalidad_georreferenciada.csv")

a_df = pd.read_csv('./accidentalidad.csv', skiprows=1,header=None,sep=';')
a_df.rename(columns={18:'fecha_hora',22: 'objectid', 23:'radicado'},inplace=True)

am_df = pd.read_csv('./accidentalidad_motos.csv')
am_df['moto']=1






In [66]:
ag_df = ag_df.sort_values(by=["radicado", "objectid"])
ag_df.head(5)

,x,y,objectid,radicado,fecha,hora,dia,periodo,clase,direccion,direccion_enc,cbml,tipo_geocod,gravedad,barrio,comuna,diseno,dia_nombre,mes,mes_nombre,x_magnamed,y_magnamed,longitud,latitud,file,shape
93458,-75.591921,6.282942,507560,12,2017/01/01 00:00:00+00,05:05:00,1,2017,Atropello,CL 77 A CR 88,CL 077 A 088 000 00000,0713,Malla vial,MUERTO,Aures No.1,Robledo,NaN,DOMINGO,1,NaN,832386.43,1186760.60,-75.591921,6.282942,accidentalidad_georreferenciada_2017.csv,NaN
93459,-75.613491,6.284049,507561,20,2017/01/01 00:00:00+00,08:30:00,1,2017,Choque,CL 66 CR 103 B - 15,CL 066 103 B 015 00000,60980380285,Nomenclatura con Interior,MUERTO,Área de Expansión Pajarito,Corregimiento de San Cristóbal,NaN,DOMINGO,1,NaN,829999.08,1186883.23,-75.613491,6.284049,accidentalidad_georreferenciada_2017.csv,NaN
223849,-75.604448,6.250820,252567,38,2014/11/25 00:00:00+00,12:33:00,25,2014,Atropello,CR 84 CL 44 - 05,CR 084 044 005 00000,1204,Malla vial,MUERTO,La América,La América,NaN,MARTES,11,NaN,830999.75,1183207.50,-75.604448,6.250820,accidentalidad_georreferenciada_2014.csv,NaN
93460,-75.703816,6.221806,507562,47,2017/01/02 00:00:00+00,01:30:00,2,2017,Choque,CR 42 CL 92 - 50,CR 042 092 050 00000,NaN,No Ubicada,MUERTO,NaN,NaN,NaN,LUNES,1,NaN,820000.00,1180000.00,-75.703816,6.221806,accidentalidad_georreferenciada_2017.csv,NaN
93462,-75.567295,6.254069,507564,499,2017/01/05 00:00:00+00,07:55:00,5,2017,Atropello,CR 51 D CL 54,CR 051 D 054 000 00000,1005,Malla vial aproximada: CR 51-54,MUERTO,Estación Villa,La Candelaria,NaN,JUEVES,1,NaN,835112.08,1183566.79,-75.567295,6.254069,accidentalidad_georreferenciada_2017.csv,NaN


In [45]:
ag_df[ag_df['radicado']=='62196']

,x,y,objectid,radicado,fecha,hora,dia,periodo,clase,direccion,direccion_enc,cbml,tipo_geocod,gravedad,barrio,comuna,diseno,dia_nombre,mes,mes_nombre,x_magnamed,y_magnamed,longitud,latitud,file,shape
132131,-75.557246,6.299648,546233,62196,2017/12/25 00:00:00+00,10:00:00,25,2017,Choque,CR 52 CL 107,CR 052 107 000 00000,0206,Malla vial,MUERTO,Santa Cruz,Andalucía,NaN,LUNES,12,NaN,836224.31,1188608.41,-75.557246,6.299648,accidentalidad_georreferenciada_2017.csv,NaN
132132,-75.557246,6.299648,546234,62196,2017/12/25 00:00:00+00,10:00:00,25,2017,Choque,CR 52 CL 107,CR 052 107 000 00000,0206,Malla vial,MUERTO,Santa Cruz,Andalucía,NaN,LUNES,12,NaN,836224.31,1188608.41,-75.557246,6.299648,accidentalidad_georreferenciada_2017.csv,NaN
132133,-75.557246,6.299648,546235,62196,2017/12/25 00:00:00+00,10:00:00,25,2017,Choque,CR 52 CL 107,CR 052 107 000 00000,0206,Malla vial,MUERTO,Santa Cruz,Andalucía,NaN,LUNES,12,NaN,836224.31,1188608.41,-75.557246,6.299648,accidentalidad_georreferenciada_2017.csv,NaN


In [47]:
ag_df.drop_duplicates(subset=["radicado"], keep="first", inplace=True)

In [49]:
ag_df[ag_df['radicado']=='62196']

,x,y,objectid,radicado,fecha,hora,dia,periodo,clase,direccion,direccion_enc,cbml,tipo_geocod,gravedad,barrio,comuna,diseno,dia_nombre,mes,mes_nombre,x_magnamed,y_magnamed,longitud,latitud,file,shape
132131,-75.557246,6.299648,546233,62196,2017/12/25 00:00:00+00,10:00:00,25,2017,Choque,CR 52 CL 107,CR 052 107 000 00000,0206,Malla vial,MUERTO,Santa Cruz,Andalucía,NaN,LUNES,12,NaN,836224.31,1188608.41,-75.557246,6.299648,accidentalidad_georreferenciada_2017.csv,NaN


In [52]:
ag_df['gravedad'].value_counts()

HERIDO         144059
SOLO DAÑOS     116379
Con heridos      7163
Solo daños       5269
MUERTO           1530
Con muertos        61
Name: gravedad, dtype: int64

In [59]:
ag_df[ag_df['objectid']==507563]


,x,y,objectid,radicado,fecha,hora,dia,periodo,clase,direccion,direccion_enc,cbml,tipo_geocod,gravedad,barrio,comuna,diseno,dia_nombre,mes,mes_nombre,x_magnamed,y_magnamed,longitud,latitud,file,shape
93461,-75.559541,6.245486,507563,5.0016e+19,2017/01/03 00:00:00+00,09:41:00,3,2017,Choque y Atropello,CR 39 CL 50,CR 039 050 000 00000,1016,Malla vial,MUERTO,Boston,La Candelaria,NaN,MARTES,1,NaN,835970.4,1182617.38,-75.559541,6.245486,accidentalidad_georreferenciada_2017.csv,NaN


In [ ]:
df.clase = df.clase.apply(lambda x: unidecode(x).upper().strip().replace(' de ',' '))

In [27]:
from ast import literal_eval
[l.lower() for l in list(literal_eval(list(a_df.loc[a_df[0]=="1.0", 4][0:1])[0]).keys())]

['@timestamp',
 'cbml',
 'expediente',
 'tipo',
 'direccion encasillada',
 'clase_accidente',
 'location',
 'mes',
 'fecha_accidentes',
 'año',
 'comuna',
 'tags',
 'direccion',
 'host',
 'fingerprint',
 'path',
 'numcomuna',
 'diseño',
 'barrio',
 'gravedad_accidente',
 'fecha_accidente',
 'y',
 'x',
 '@version',
 'id',
 'nro_radicado']

In [39]:
a_df = a_df.sort_values(by=['radicado', 'objectid'])
a_df['radicado'].value_counts()


1613593    1
1620992    1
1645994    1
1567028    1
1465342    1
          ..
1451850    1
1442267    1
1635939    1
1436460    1
1466846    1
Name: radicado, Length: 261875, dtype: int64

In [29]:
a_df.drop_duplicates(subset=['radicado'], keep="first", inplace=True)

In [30]:
am_df = am_df.sort_values(by=['NRO_RADICADO'])
am_df['NRO_RADICADO'].value_counts()

1647286    2
1638618    2
1641667    2
1647440    2
1640023    2
          ..
1578967    1
1603547    1
1593308    1
1591261    1
1574913    1
Name: NRO_RADICADO, Length: 136653, dtype: int64

In [31]:
ag_df = ag_df[~ag_df.radicado.isna()]
a_df=a_df[~a_df.radicado.isna()]
dtype = dict(radicado=str)
ag_df.radicado = ag_df.radicado.astype(dtype)
a_df.radicado = a_df.radicado.astype(dtype)
ag_df.radicado = ag_df.radicado.str.strip()
ag_df.radicado = ag_df.radicado.str.replace("(\d+)(.0)*","\\1") #.dtype #.sort_values(by="radicado", ascending=False)
ag_df.radicado = ag_df.radicado.str.replace("^(0+)(\d+)", "\\2")
a_df.radicado = a_df.radicado.str.strip()
a_df.radicado = a_df.radicado.str.replace("(\d+)(.0)*","\\1")

In [32]:
am_df.drop_duplicates(subset=['NRO_RADICADO'], keep="first", inplace=True)

Merge accidentes geo with accidentes, and motos

In [33]:
ag_df = ag_df.merge(a_df[['fecha_hora','radicado']],on='radicado',how='left')

In [34]:
am_df.rename(columns={'NRO_RADICADO':'radicado'},inplace=True)

ag_df = ag_df.merge(am_df[['radicado','moto']],on='radicado',how='left')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [155]:
import numpy as np
ag_df['fecha'] = ag_df['fecha'].apply(lambda x : x.replace(' 00:00:00+00',''))
ag_df['fecha'] = pd.to_datetime(ag_df['fecha'],format='%Y/%m/%d')

ag_df[ag_df['fecha'].dt.year>2014]['moto'].fillna(0,inplace=True)



In [156]:
ag_df[ag_df['radicado'].isin(am_df['radicado'])].groupby(pd.Grouper(key='fecha',freq='Y')).count()

,x,y,objectid,radicado,hora,dia,periodo,clase,direccion,direccion_enc,cbml,tipo_geocod,gravedad,barrio,comuna,diseno,dia_nombre,mes,mes_nombre,x_magnamed,y_magnamed,longitud,latitud,file,shape,fecha_hora,moto
fecha,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-12-31,19441,19441,19441,19441,19441,19441,19441,19441,19441,19240,17471,19441,19441,17471,17471,19441,19441,19441,0,19441,19441,19441,19441,19441,0,0,19441
2016-12-31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-12-31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2018-12-31,24596,24596,24596,24596,24596,24596,24596,24596,24596,24517,22701,24596,24596,22692,22701,24596,24596,24596,0,24596,24596,24596,24596,24596,0,0,24596
2019-12-31,19246,19246,26804,26804,26804,26804,26804,26804,26804,26765,25216,26804,26804,24718,25915,26804,26804,26804,26804,26804,26804,26804,26804,26804,0,4271,26804


## How did you merge the two DataFrames (aggregated data from 2014-2019 and accidentalidad.csv)

In [28]:
print(a_df.isna().any())
print(ag_df.isna().any())
print(am_df.isna().any())


0           False
1            True
2            True
3           False
4            True
5            True
6            True
7            True
8            True
9            True
10           True
11           True
12           True
13           True
14           True
15           True
16           True
17           True
fecha        True
19           True
20           True
21           True
22           True
radicado     True
dtype: bool
x                 True
y                 True
objectid         False
radicado          True
fecha            False
hora             False
dia              False
periodo          False
clase             True
direccion        False
direccion_enc     True
cbml              True
tipo_geocod      False
gravedad         False
barrio            True
comuna            True
diseno            True
dia_nombre       False
mes              False
mes_nombre        True
x_magnamed       False
y_magnamed       False
longitud         False
latitud          False
file

In [112]:
ag_df[ag_df.radicado.isna()] # check the nulls

,x,y,objectid,radicado,fecha,hora,dia,periodo,clase,direccion,direccion_enc,cbml,tipo_geocod,gravedad,barrio,comuna,diseno,dia_nombre,mes,mes_nombre,x_magnamed,y_magnamed,longitud,latitud,file,shape
39496,-75.561594,6.293715,546224,NaN,2017/12/21 00:00:00+00,09:58:00,21,2017,Atropello,CR 52 CL 100 - 75,CR 052 100 075 00000,02100090113,Nomenclatura con Interior,MUERTO,Santa Cruz,Santa Cruz,NaN,JUEVES,12,NaN,835743.14,1187952.15,-75.561594,6.293715,Accidentalidadgeorreferenciada2017.csv,NaN
40150,-75.578129,6.247262,546941,NaN,2017/09/06 00:00:00+00,05:45:00,6,2017,Atropello,CL 44 entre CR 59 y 60,"CL 044 59, 060 00000",1008,Malla vial,MUERTO,La Candelaria,Corazón de Jesús,NaN,MIÉRCOLES,9,NaN,833912.85,1182813.77,-75.578129,6.247262,Accidentalidadgeorreferenciada2017.csv,NaN
37075,-75.564198,6.261494,683301,NaN,2018/05/07 00:00:00+00,06:00:00,7,2018,Choque,CR 51 CL 64,CR 051 064 000 00000,1001,Malla vial,MUERTO,Prado,La Candelaria,NaN,LUNES,5,NaN,835454.94,1184388.09,-75.564198,6.261494,Accidentalidadgeorreferenciada2018.csv,NaN
37076,-75.546095,6.271341,683302,NaN,2018/11/01 00:00:00+00,07:40:00,1,2018,Choque,CL 80 CR 32 - 82,CL 080 032 082 00000,03040160010,Catastro con Interior,MUERTO,Santa Inés,Manrique,NaN,JUEVES,11,NaN,837458.56,1185477.35,-75.546095,6.271341,Accidentalidadgeorreferenciada2018.csv,NaN
37077,-75.593598,6.270184,683303,NaN,2018/12/26 00:00:00+00,12:30:00,26,2018,Choque,CL 59 CR 78 AA,CL 059 078 AA 000 00000,1114,Malla vial,MUERTO,Los Colores,Laureles Estadio,NaN,MIÉRCOLES,12,NaN,832200.75,1185349.38,-75.593598,6.270184,Accidentalidadgeorreferenciada2018.csv,NaN


In [113]:
ag_df.clase.value_counts(dropna=False)

Choque                170276
Otro                   26585
Atropello              24647
Caida Ocupante         16419
Volcamiento             8249
Caída de Ocupante       5665
Incendio                  32
Caída Ocupante            16
NaN                        6
Choque                     3
Choque y Atropello         1
Name: clase, dtype: int64

In [114]:
ag_df.cbml.value_counts(dropna=False)

1019           6182
0517           5246
1507           5153
1012           4988
1105           4461
               ... 
80004340062       1
14220240062       1
15100350002       1
02030120001       1
11140030016       1
Name: cbml, Length: 712, dtype: int64

In [115]:
ag_df.radicado.value_counts()[0:21]

62196                    3
1618580.0                2
39226                    2
1662064                  2
1640023.0                2
31435                    2
1656231                  2
1639346.0                2
1639954.0                2
58560                    2
1647440.0                2
58864                    2
1647286.0                2
1655403                  2
48996                    2
050016000206201523690    2
1640118.0                2
1640922.0                2
1640356.0                2
1638618.0                2
1689154                  1
Name: radicado, dtype: int64

In [116]:
ag_df[ag_df.radicado==1638618.0]

,x,y,objectid,radicado,fecha,hora,dia,periodo,clase,direccion,direccion_enc,cbml,tipo_geocod,gravedad,barrio,comuna,diseno,dia_nombre,mes,mes_nombre,x_magnamed,y_magnamed,longitud,latitud,file,shape
21533,-75.585424,6.241112,666622,1.63862e+06,2018/08/14 00:00:00+00,05:35:00,14,2018,Choque,CR 66 CL 34 A,CR 066 034 A 000 00000,1105,Malla vial aproximada: CR 66-35,SOLO DAÑOS,Los Conquistadores,Laureles Estadio,Tramo de via,MARTES,8,NaN,833105.42,1182133.61,-75.585424,6.241112,Accidentalidadgeorreferenciada2018.csv,NaN
36260,-75.585424,6.241112,682426,1.63862e+06,2018/10/14 00:00:00+00,05:35:00,14,2018,Choque,CR 66 CL 34 A,CR 066 034 A 000 00000,1105,Malla vial aproximada: CR 66-35,SOLO DAÑOS,Los Conquistadores,Laureles Estadio,Tramo de via,DOMINGO,10,NaN,833105.42,1182133.61,-75.585424,6.241112,Accidentalidadgeorreferenciada2018.csv,NaN


In [117]:
a_df.radicado.value_counts(dropna=False)[0:30]

NaN          7
1687153.0    3
1687280.0    3
1689208.0    3
1687560.0    3
1687158.0    3
1687159.0    3
1687162.0    3
1687163.0    3
62196        3
1687164.0    3
1687165.0    3
1689064.0    3
1687166.0    3
1685159.0    3
1687167.0    3
1687169.0    3
1687154.0    3
1685140.0    3
1687171.0    3
1687292.0    3
1685517.0    3
1687136.0    3
1685493.0    3
1687138.0    3
1687557.0    3
1684827.0    3
1687140.0    3
1687144.0    3
1688636.0    3
Name: radicado, dtype: int64

In [118]:
df[df.radicado=="1638618"]

,timestamp,cbml,expediente,tipo_geocode,direccion_enc,clase,x_long_y_lat,mes,periodo,comuna,incidente,direccion,operador,path,num_comuna,diseno,barrio,gravedad,fecha,y_magnamed,x_magnet,version,id,radicado
205150,2019-06-04T20:55:57.306Z,1105,A000846486,Malla vial aproximada: CR 66-35,CR 066 034 A 000 00000,Choque,"[-75.5854551105, 6.24105317866]",8.0,2018.0,Laureles Estadio,[u'incidentes'],CR 66 CL 34 A,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,11,Tramo de via,Los Conquistadores,Solo daños,2018-08-14T22:35:00.000Z,1182133.61,833105.42,1.0,211762.0,1638618
213113,2019-06-04T20:56:23.935Z,1105,A000846486,Malla vial aproximada: CR 66-35,CR 066 034 A 000 00000,Choque,"[-75.5854551105, 6.24105317866]",10.0,2018.0,Laureles Estadio,[u'incidentes'],CR 66 CL 34 A,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,11,Tramo de via,Los Conquistadores,Solo daños,2018-10-14T22:35:00.000Z,1182133.61,833105.42,1.0,219306.0,1638618


In [119]:
df[df.radicado=="1638617"]

,timestamp,cbml,expediente,tipo_geocode,direccion_enc,clase,x_long_y_lat,mes,periodo,comuna,incidente,direccion,operador,path,num_comuna,diseno,barrio,gravedad,fecha,y_magnamed,x_magnet,version,id,radicado
203433,2019-06-04T20:55:58.641Z,402,A000846480,Malla vial,CR 051 092 000 00000,Choque,"[-75.5611117649, 6.28348983493]",8.0,2018.0,Aranjuez,[u'incidentes'],CR 51 CL 92,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,04,Tramo de via,San Isidro,Solo daños,2018-08-15T20:00:00.000Z,1186820.47,835813.52,1.0,211882.0,1638617


We found some duplicates values in "radicado" field so we procedure to drop them.

In [120]:
df = df.sort_values(by=["radicado", "id"])
df.drop_duplicates(subset=["radicado"], keep="first", inplace=True)

In order to have date and time in different columns we decided to derivate new columns from "fecha" and have separet columns for the date and the hour.

In [121]:
df["fecha_incidente"] = df.fecha.str[0:10]
df["hora_incidente"] = df.fecha.str[11:13]

In [122]:
# drop duplicates by "radicado", "objectid"
agg_df = agg_df.sort_values(by=["radicado", "objectid"])
agg_df.drop_duplicates(subset=["radicado"], keep="first", inplace=True)
agg_df = agg_df[(~agg_df.radicado.isna()) & (agg_df.radicado != "0")]

In [123]:
df.radicado

262153    1.67637e+06
262150    1.67644e+06
262168    1.67646e+06
262162    1.67646e+06
262160    1.67647e+06
             ...     
143894           9769
9909             9778
12090            9780
183864           9982
172418            NaN
Name: radicado, Length: 261876, dtype: object

In [124]:
df[df.radicado.isna()] # check the number of nulls

,timestamp,cbml,expediente,tipo_geocode,direccion_enc,clase,x_long_y_lat,mes,periodo,comuna,incidente,direccion,operador,path,num_comuna,diseno,barrio,gravedad,fecha,y_magnamed,x_magnet,version,id,radicado,fecha_incidente,hora_incidente
172418,2019-06-04T20:54:26.357Z,1008,A000665813,Malla vial,"CL 044 59, 060 00000",Atropello,"[-75.5781789259, 6.24722232125]",9.0,2017.0,La Candelaria,[u'incidentes'],CL 44 entre CR 59 y 60,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,10,NaN,Corazón de Jesús,Con muertos,2017-09-06T10:45:00.000Z,1182813.77,833912.85,1.0,185160.0,NaN,2017-09-06,10


In [125]:
agg_df = agg_df[~agg_df.radicado.isna()]
df=df[~df.radicado.isna()]
dtype = dict(radicado=str)
agg_df.radicado = agg_df.radicado.astype(dtype)
df.radicado = df.radicado.astype(dtype)
agg_df.radicado = agg_df.radicado.str.strip()
agg_df.radicado = agg_df.radicado.str.replace("(\d+)(.0)*","\\1") #.dtype #.sort_values(by="radicado", ascending=False)
agg_df.radicado = agg_df.radicado.str.replace("^(0+)(\d+)", "\\2")
df.radicado = df.radicado.str.strip()
df.radicado = df.radicado.str.replace("(\d+)(.0)*","\\1")

In [126]:
df.fecha.max()

'2019-10-01T04:20:00.000Z'

In [131]:
df.shape
df
# r = df[(df.cbml.isna()) & (df.x_long_y_lat == '[-75.7037762763, 6.22141524356]') ][['x_long_y_lat','barrio','comuna','direccion','direccion_enc','cbml']]
# r

,timestamp,cbml,expediente,tipo_geocode,direccion_enc,clase,x_long_y_lat,mes,periodo,comuna,incidente,direccion,operador,path,num_comuna,diseno,barrio,gravedad,fecha,y_magnamed,x_magnet,version,id,radicado,fecha_incidente,hora_incidente
262153,2019-11-07T16:07:39.311Z,103,A001002056,Malla vial,CR 043 A 110 B 000 00000,Atropello,"[-75.5478922382, 6.29966416516]",6.0,2019.0,Popular,[u'incidentes'],CR 43 A CL 110 B,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,01,Tramo de via,Popular,Con heridos,2019-07-01T02:50:00.000Z,1.188606e+06,837281.840000,1.0,10647.0,1676369,2019-07-01,02
262150,2019-11-07T16:07:39.311Z,1102,A001002214,Malla vial,CR 066 048 D 000 00000,Choque,"[-75.5829966564, 6.25459513351]",6.0,2019.0,Laureles Estadio,[u'incidentes'],CR 66 CL 48 D,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,11,Lote o Predio,Suramericana,Solo daños,2019-07-01T01:40:00.000Z,1.183631e+06,833381.860000,1.0,10641.0,1676437,2019-07-01,01
262168,2019-11-07T16:07:39.312Z,607,A001002310,Malla vial,CR 083 091 A 000 00000,Choque,"[-75.5849391248, 6.28861482868]",7.0,2019.0,Doce de Octubre,[u'incidentes'],CR 83 CL 91 A,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,06,Tramo de via,Kennedy,Solo daños,2019-07-01T23:20:00.000Z,1.187395e+06,833177.650000,1.0,10699.0,1676456,2019-07-01,23
262162,2019-11-07T16:07:39.312Z,312,A001002302,Malla vial,CR 023 B 084 B 000 00000,Choque,"[-75.5378038865, 6.27686386215]",7.0,2019.0,Manrique,[u'incidentes'],CR 23 B CL 84 B,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,03,Tramo de via,Oriente,Solo daños,2019-07-01T20:30:00.000Z,1.186080e+06,838391.450000,1.0,10686.0,1676461,2019-07-01,20
262160,2019-11-07T16:07:39.312Z,NaN,A001002169,No Ubicada,CR 067 025 000 00000,Choque,"[-75.7037762763, 6.22141524356]",7.0,2019.0,Sin Inf,[u'incidentes'],CR 67 CL 25,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,No ubicada,Tramo de via,NaN,Con heridos,2019-07-01T16:40:00.000Z,1.180000e+06,820000.000000,1.0,10674.0,1676468,2019-07-01,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12089,2019-06-04T20:44:08.013Z,3.02021e+09,A1485619,Catastro con Interior,CR 039 085 A 035 00000,Choque,"[-75.5477362391, 6.27745474338]",2.0,2014.0,Manrique,[u'incidentes'],CR 39 CL 85 A - 35,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,03,NaN,Las Granjas,Con muertos,2014-02-24T08:03:00.000Z,1.186149e+06,837292.200000,1.0,6512.0,9766,2014-02-24,08
143894,2019-06-04T20:53:18.109Z,6.08101e+09,A000497560,Catastro con Interior,CR 083 101 A 021 00000,Atropello,"[-75.5821690996, 6.30114194037]",2.0,2017.0,Doce de Octubre,[u'incidentes'],CR 83 CL 101 A - 21,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,06,NaN,Picacho,Con muertos,2017-02-12T01:30:00.000Z,1.188780e+06,833488.260000,1.0,165498.0,9769,2017-02-12,01
9909,2019-06-04T20:44:08.014Z,1.50304e+10,A1485792,Catastro con Interior,CL 025 065 C 050 00000,Atropello,"[-75.5855316677, 6.22630141567]",2.0,2014.0,Guayabal,[u'incidentes'],CL 25 CR 65 C - 50,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,15,NaN,Trinidad,Con muertos,2014-02-24T12:30:00.000Z,1.180502e+06,833092.280000,1.0,6513.0,9778,2014-02-24,12
12090,2019-06-04T20:44:08.014Z,1.62002e+10,A1485790,Nomenclatura con Interior,CL 033 078 177 00000,Atropello,"[-75.6000288989, 6.23861225192]",2.0,2014.0,Belén,[u'incidentes'],CL 33 CR 78 - 177,Operador2,C:/ELK/logstash-6.5.1/data/Medata/Incidentes/I...,16,NaN,El Nogal-Los Almendros,Con muertos,2014-02-24T12:15:00.000Z,1.181868e+06,831491.290000,1.0,6515.0,9780,2014-02-24,12


In [128]:
df.clase = df.clase.apply(lambda x: unidecode(x).lower().strip().replace(' de ',' '))

AttributeError: 'float' object has no attribute 'encode'

In [129]:
df.to_csv("../processed/preprocessed_accidents.csv",index=False)